In [3]:
from transformers import MarianMTModel, MarianTokenizer

# 사전 학습된 MarianMT 모델 (예제: 한국어 ↔ 영어
model_name = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate(text):
    tokens = tokenizer(text, return_tensors = "pt", padding = True, truncation = True)
    translated_tokens = model.generate(**tokens)
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens = True)
    return translated_text[0]

translated_text = translate("나는 오늘 학교에 가서 친구를 만났어")
print(translated_text)

I went to school today and I met a friend.


In [4]:
from deep_translator import GoogleTranslator

# 한국어 → 영어 번역
text = "나는 오늘 학교에 가서 친구를 만났어."
translated_text = GoogleTranslator(source='ko', target='en').translate(text)
print(translated_text)  # "I went to school today and met my friend."

# 영어 → 미국수어(ASL) 번역 (Google API에서는 완전한 지원이 안 될 수 있음)
asl_translation = GoogleTranslator(source='en', target='sgn-US').translate(translated_text)
print(asl_translation)  # ASL 스타일 문장 출력 가능성 있음


I went to school today and met a friend.


LanguageNotSupportedException: sgn-US --> No support for the provided language.
Please select on of the supported languages:
{'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'assamese': 'as', 'aymara': 'ay', 'azerbaijani': 'az', 'bambara': 'bm', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bhojpuri': 'bho', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dhivehi': 'dv', 'dogri': 'doi', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'ewe': 'ee', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'guarani': 'gn', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic

In [40]:
import stanza
import re

# Stanza 한국어 모델 다운로드 (최초 실행 시 필요)
stanza.download("ko")
nlp = stanza.Pipeline("ko", processors="tokenize,pos,lemma,depparse")

# ✅ 모든 조사(은, 는, 이, 가, 를, 을 등)를 자동 제거하는 함수
def remove_josa(word):
    """
    한국어에서 명사(NOUN) 뒤에 붙은 조사(은, 는, 이, 가, 를, 을 등)를 자동 제거
    """
    return re.sub(r"(은|는|이|가|를|을)$", "", word)

# ✅ 동사 원형 변환을 위한 후처리 함수
def refine_verb(verb):
    verb = re.sub(r"고$", "", verb)  # "고" 제거
    verb = re.sub(r"어$", "", verb)  # "어" 제거
    verb = re.sub(r"아$", "", verb)  # "아" 제거
    return verb + "다"  # 원형 복원

# ✅ 불필요한 조사 및 "+" 문자 제거 함수
def clean_up_sentence(sentence):
    sentence = re.sub(r"\+", "", sentence)  # "+" 문자 완전히 제거
    sentence = re.sub(r"\s+", " ", sentence).strip()  # 불필요한 공백 정리
    sentence = re.sub(r"\s*에서\s*", " ", sentence)  # "에서" 제거
    sentence = re.sub(r"\s*에\s*", " ", sentence)  # "에" 제거
    return sentence

# ✅ 수어체 변환 함수 (주어 유지 + 조사 자동 제거)
def translate_to_sign_language(text):
    doc = nlp(text)  # 형태소 분석 수행
    sign_sentence = []

    for sentence in doc.sentences:
        words = []
        verbs = []
        removed_words = ["PART", "PUNCT", "SCONJ", "AUX", "CCONJ", "ADP", "DET"]  # 불필요한 조사, 부사, 접속사 제거

        for word in sentence.words:
            if word.upos in removed_words:
                continue

            # ✅ 명사(NOUN)에서 조사 제거 (예: "고양이가" → "고양이", "책을" → "책")
            if word.upos == "PRON" or word.upos == "NOUN":
                words.append(remove_josa(word.text))  # 조사 제거된 단어 추가
                continue

            # "~고 싶다" 변환: "원하다"로 변경
            if word.text in ["싶다"] and verbs:
                verbs.append("원하다")
                continue

            # "안/않다" 같은 부정어 처리
            if word.text in ["안", "않다"]:
                verbs.append("아니다")
                continue

            # ✅ 동사 변환 (예: "배우고" → "배우다")
            if word.upos == "VERB":
                lemma = refine_verb(word.lemma) if word.lemma else refine_verb(word.text)
                verbs.append(lemma)  # 동사 원형 변환
            else:
                # ✅ 조사 제거 (은, 는, 이, 가, 를, 을 등을 자동 제거)
                words.append(remove_josa(word.text))

        # ✅ 최종 변환된 문장 추가
        words_sov = words + verbs  # 동사를 문장 끝으로 이동
        transformed_sentence = " ".join(words_sov) + "."
        cleaned_sentence = clean_up_sentence(transformed_sentence)  # 최종 정리된 문장
        
        sign_sentence.append(cleaned_sentence)

    return " ".join(sign_sentence)

# ✅ 테스트 문장
test_sentences = [
    "나는 지금 집에 가고 싶어.",
    "내일 영화 보고 싶어.",
    "저는 집에서 밥을 먹고 싶어요.",
    "고양이가 창문 위에서 자고 있어.",
    "나는 한국어를 배우고 싶지 않아.",
    "나는 새로운 핸드폰을 사고 싶어.",
    "그는 책을 읽고 있어.",
    "문문은 밥을 먹고 있어."
]

for sentence in test_sentences:
    translated_text = translate_to_sign_language(sentence)
    print(f"원본 문장: {sentence}")
    print(f"수어체 변환: {translated_text}\n")


2025-03-14 17:46:19 INFO: Downloaded file to /home/usou/stanza_resources/resources.json
2025-03-14 17:46:19 INFO: Downloading default packages for language: ko (Korean) ...
2025-03-14 17:46:20 INFO: File exists: /home/usou/stanza_resources/ko/default.zip
2025-03-14 17:46:21 INFO: Finished downloading models and saved to /home/usou/stanza_resources
2025-03-14 17:46:21 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-03-14 17:46:21 INFO: Downloaded file to /home/usou/stanza_resources/resources.json
2025-03-14 17:46:22 INFO: Loading these models for language: ko (Korean):
| Processor | Package        |
------------------------------
| tokenize  | kaist          |
| pos       | kaist_nocharlm |
| lemma     | kaist_nocharlm |
| depparse  | kaist_nocharlm |

2025-03-14 17:46:22 INFO: Using device: cuda
2025-03-14 17:46:22 INFO: Loading: tokeniz

원본 문장: 나는 지금 집에 가고 싶어.
수어체 변환: 나 지금 집 가다.

원본 문장: 내일 영화 보고 싶어.
수어체 변환: 내일 영화 보다.

원본 문장: 저는 집에서 밥을 먹고 싶어요.
수어체 변환: 저 집 밥 먹다.

원본 문장: 고양이가 창문 위에서 자고 있어.
수어체 변환: 고양이 창문 위 자다.

원본 문장: 나는 한국어를 배우고 싶지 않아.
수어체 변환: 나 한국어 배우다.

원본 문장: 나는 새로운 핸드폰을 사고 싶어.
수어체 변환: 나 새로운 핸드폰 사다.

원본 문장: 그는 책을 읽고 있어.
수어체 변환: 그 책 읽다.

원본 문장: 문문은 밥을 먹고 있어.
수어체 변환: 문문 밥 먹다.



In [34]:
# 테스트 문장
test_sentences = [
  "문문은 지금 열심히 작업을 하고 있어요"
]

for sentence in test_sentences:
    translated_text = translate_to_sign_language(sentence)
    print(f"원본 문장: {sentence}")
    print(f"수어체 변환: {translated_text}\n")


원본 문장: 문문은 지금 열심히 작업을 하고 있어요
수어체 변환: 문문은 지금 열심히 작업 하다.



# STT

In [3]:
import pyaudio
import wave
import whisper
from transformers import pipeline
import numpy as np

# 마이크 설정
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000  # Whisper 모델과 호환되는 샘플 레이트
CHUNK = 1024
RECORD_SECONDS = 2  # 녹음 시간
WAVE_OUTPUT_FILENAME = "recorded_audio.wav"

# 감정 분석 모델 로드 (Hugging Face)
emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

def record_audio():
    """마이크에서 음성을 녹음하고 파일로 저장"""
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    print("🎤 음성 녹음 시작... ({}초 동안)".format(RECORD_SECONDS))
    
    frames = []
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("🔴 녹음 종료")

    # 스트림 닫기
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # WAV 파일로 저장
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    return WAVE_OUTPUT_FILENAME

def transcribe_audio(file_path):
    """녹음된 오디오 파일을 Whisper를 이용해 텍스트로 변환"""
    model = whisper.load_model("small")  # Whisper 모델 (tiny, base, small, medium, large 선택 가능)
    result = model.transcribe(file_path)
    return result["text"]

def analyze_emotion(text):
    """텍스트를 감정 분석 모델에 입력하여 감정을 분석"""
    analysis = emotion_model(text)
    return analysis

# 실행 흐름
if __name__ == "__main__":
    audio_file = record_audio()
    transcribed_text = transcribe_audio(audio_file)
    print(f"\n📝 STT 변환 결과: {transcribed_text}")

    emotion_result = analyze_emotion(transcribed_text)
    print("\n📊 감정 분석 결과:")
    for emotion in emotion_result[0]:  # 가장 높은 감정 선택
        print(f"  {emotion['label']}: {emotion['score']:.4f}")



Device set to use cuda:0
/home/usou/venv/superbad/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmRe

🎤 음성 녹음 시작... (2초 동안)
🔴 녹음 종료

📝 STT 변환 결과:  වවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවව

📊 감정 분석 결과:
  anger: 0.1190
  disgust: 0.0434
  fear: 0.2542
  joy: 0.0237
  neutral: 0.4818
  sadness: 0.0639
  surprise: 0.0141
